# **Feature Generation**

Protein Sequence to 1280 long features using ESM 1b



In [ ]:
'''pytorch 1.7.1 along with cuda 10.1 is required for the package to run!
so install it using the cell below'''

In [ ]:
!pip install torch==1.7.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 15 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.1+cu101 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.7.1+cu101 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.7.1+cu101 which is incompatible.


In [ ]:
import torch 
print(torch.__version__)

1.7.1+cu101


Install Bio_enbeddings
[More Info can be found on their GitHub page](https://github.com/sacdallago/bio_embeddings.git)

The cell below will take approximately 5-10 mins

In [ ]:
!pip3 install -U pip > /dev/null
!pip3 install -U "bio-embeddings[all] @ git+https://github.com/sacdallago/bio_embeddings.git" > /dev/null

  Running command git clone --filter=blob:none --quiet https://github.com/sacdallago/bio_embeddings.git /tmp/pip-install-fj5ksknp/bio-embeddings_9668d98a96dd4c9cb2c4addfd1ab38bb
tcmalloc: large alloc 1102381056 bytes == 0x36d8e000 @  0x7fa5102eb615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x549576 0x593fce 0x511e2c 0x593dd7 0x511e2c 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x511e2c 0x549576 0x593fce
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.2 which is incompatible.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.9.1 which is incom

Mounting drive to read in the training data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

# *the cell below will give an error when you run it for the first time but ignore the error and re run the same cell again and that error will go away!*

In [ ]:
from bio_embeddings.embed import ESM1bEmbedder

Reading the training data

In [ ]:
d = pd.read_csv('/content/drive/MyDrive/Enzyme_Dataset/Train.csv')
d.head()

,SEQUENCE_ID,SEQUENCE,CREATURE,LABEL
0,00011UBJ,MPWQELSITVPHEYVEPISYLFGRYGKGVSTELAGNGQVLLRTYLT...,creature9,class11
1,0001HZ3P,MGKLVIGSRGSELALWQANHIKERLKKECLIESEIQIVKTKGDKIL...,creature3,class13
2,0003NJDO,MFIVAVLMLAFLIFVHELGHFTIARICGVKVEVFSIGFGKKLCFFK...,creature3,class16
3,0003XVE3,MKNTSLSTTLTMNDTIAAIATPLGKGAISIIKISGHNALNILKQLT...,creature3,class12
4,00049BS0,MGNRLTKIYTKTGDSGLTGLADGSRTEKDSLRIFVIGDVDELNSLL...,creature8,class13


*apparently the esm1b embedder cannot process sequeces of length greater than 1024 hen we need to truncate it to a value less than 1022(reserving 2 for seperator and unk tokens). I chose 1000*

In [ ]:
def truncator(s):
  if len(str(s)) < 1000:
    ans = s
  else:
    ans = str(s)[:1000]
  return ans


d['SEQUENCE'] = d['SEQUENCE'].apply(truncator)


In [ ]:
sequencez = d['SEQUENCE'].values

In [ ]:
embedder = ESM1bEmbedder(device = 'cuda')

*storing the 1280 lomg embedding for each protein sequence in a list called "embeddings" and then dumping it into a pickel file for avoiding to generate it again and again*

*The cell below will take around 24-30 hours to process all of the 858777 training sequences. (I will provide a link for you to download it using wget to avoid this step)*

In [ ]:
embeddings = []
for sequence in sequencez:
  per_amino = embedder.embed(sequence)
  embeddings.append(embedder.reduce_per_protein(per_amino))

In [ ]:
import pickle
with open('ESM1b_train_embeddings.pkl', 'wb') as f:
  pickle.dump(embeddings, f)

# The same code and technique can be used to extract the embeddings of the sequences from Test.csv

In [ ]:
d = pd.read_csv('/content/drive/MyDrive/Enzyme_Dataset/Test.csv')
d.head()

,SEQUENCE_ID,SEQUENCE,CREATURE,LABEL
0,00011UBJ,MPWQELSITVPHEYVEPISYLFGRYGKGVSTELAGNGQVLLRTYLT...,creature9,class11
1,0001HZ3P,MGKLVIGSRGSELALWQANHIKERLKKECLIESEIQIVKTKGDKIL...,creature3,class13
2,0003NJDO,MFIVAVLMLAFLIFVHELGHFTIARICGVKVEVFSIGFGKKLCFFK...,creature3,class16
3,0003XVE3,MKNTSLSTTLTMNDTIAAIATPLGKGAISIIKISGHNALNILKQLT...,creature3,class12
4,00049BS0,MGNRLTKIYTKTGDSGLTGLADGSRTEKDSLRIFVIGDVDELNSLL...,creature8,class13


In [ ]:
sequencez = d['SEQUENCE'].values

In [ ]:
embeddings = []
for sequence in sequencez:
  per_amino = embedder.embed(sequence)
  embeddings.append(embedder.reduce_per_protein(per_amino))

In [ ]:
import pickle
with open('ESM1b_testing_embeddings.pkl', 'wb') as f:
  pickle.dump(embeddings, f)

Now that we have our embeddings for both train.csv as well as Test.csv, we can proceed towards training them using a Machine learning model.
Please see the Training notebook for that!



---

